Evaluate different ising influence strategies on the Pokec dataset for a range of different linear field gradients.

Created on: 28/09/20

In [1]:
import networkx as nx
import numpy as np
import pandas as pd
import ast
import seaborn as sns
import matplotlib.pyplot as plt
import tqdm
import random
import itertools
import matplotlib
from scipy import stats
import math

from ising_block_level_influence import N_Block_sbm_class as NBlock
from ising_block_level_influence import projection_simplex as proj
from ising_block_level_influence import mean_field_IIM
from spatial_spin_monte_carlo import spatial_spin_monte_carlo as Spins
import Pokec_processing as  PokProc

from pokec_utils import *

### Read in processed data

This data has been generated using:

1. **make_Pokec_reduced_profiles.ipynb** - which then feeds data into:

2. The pre-processing script **'make_bratislava_graph_and_blocks.py'**.

In [2]:
graph = nx.read_graphml('Data/Bratislava_graph.graphml')
beta_c = Spins.crit_beta_sparse(graph)
bratislava_profiles = pd.read_csv("Data/bratislava_profiles.csv")
coupling_graph = nx.read_graphml('Data/Bratislava_coupling.graphml')
block_data = pd.read_csv('Data/block_info.csv',converters={'Block' : ast.literal_eval})
mean_block_ages = list(block_data['average_age'])
block_sizes = list(block_data['block_size'])
block_names = list(block_data['block_name'])
block_data['age_group'] = [ 'ages_' + k.split('_')[-1] for k in list(block_data['block_name'])]
block_data.head()

Computing critical temperature


,Unnamed: 0,block_name,block_size,average_age,age_group
0,0,Okolie_ages_1-17,1234,9.0,ages_1-17
1,1,Okolie_ages_18-21,1939,19.5,ages_18-21
2,2,Okolie_ages_22-28,3154,25.0,ages_22-28
3,3,Okolie_ages_29-112,3458,70.5,ages_29-112
4,4,Petrzalka_ages_1-17,1398,9.0,ages_1-17


In [3]:
def linear_field(x : np.ndarray,gradient :float) :
    return gradient*x
#Scale ages to [-1,1]:
rescaled_ages = [ (k-np.mean(mean_block_ages))/(max(mean_block_ages)-min(mean_block_ages)) for k in mean_block_ages ]

In [4]:
bratislava_profiles_indices = bratislava_profiles.reset_index()
groups = [ bratislava_profiles_indices.loc[bratislava_profiles_indices['block']==block] for block in block_names]
groups_node_ids = [list(k['index']) for k in groups]

## Sweep over $\beta$ values

In [6]:
def check_group(input_val,group_label) :
    if input_val == group_label :
        return 1.0
    else :
        return 0.0
    
def mean_and_se(values) :
    return f"{np.mean(values)} +/- {stats.sem(values)} "

#Seed the random number generators:
seed = 1
random.seed(seed)
np.random.seed(seed)

Field_Budget = 2500.0
T=10000
T_Burn = 2*300000
Samples = 15
sample_frac = 1.0 # for snapshot control
eval_data = pd.DataFrame()
control_data = pd.DataFrame()

init_sign=1.0

#beta_factor_vals = [1.0,3.0,5.0,7.0,9.0]
#beta_factor_vals=[2.0]
grad_vals =  np.arange(-10.0,12.0,2.0)

con_names = ['no con','unif','full','block','sv','nc','snapshot']
control_dict={}

for gradient in tqdm.tqdm_notebook(grad_vals) : 
    
    age_field = [linear_field(a,gradient) for a in rescaled_ages ]
    age_field_map = {k:j for k,j in zip(list(block_data['age_group']),age_field)}

    # Block MF setup:
    coupling_graph = nx.convert_node_labels_to_integers(coupling_graph)
    block_graph_ising_system = mean_field_IIM.mean_field_ising_system(coupling_graph,age_field)
    block_graph_ising_system.gamma = 1.0
    block_graph_ising_system.tol = 1E-5
    block_graph_ising_system.max_mf_fp_iterations = 10000
    block_graph_ising_system.mf_fp_init_state = init_sign*np.ones(len(block_sizes))
    block_graph_ising_system.mf_fp_noisy = False

    # Full graph Mf setup
    background_field = np.asarray([age_field_map[k] for k in list(bratislava_profiles['age_group'])])
    relab_graph = nx.relabel.convert_node_labels_to_integers(graph)
    full_graph_ising_system = mean_field_IIM.mean_field_ising_system(relab_graph,background_field)
    full_graph_ising_system.mf_fp_init_state = init_sign*np.ones(len(graph))
    full_graph_ising_system.mf_fp_noisy = False


    for beta_factor in tqdm.tqdm_notebook(beta_factor_vals) : 

        beta = beta_c*beta_factor

        # Block gradient computation: 
        m_block = block_graph_ising_system.mf_magnetization(age_field,beta)
        mag_gradient = block_graph_ising_system.mf_magnetization_gradient(m_block, beta)
        mag_grad_map = { i:j for i,j in zip(block_names,mag_gradient)}
        block_names_list = list(bratislava_profiles['block'])
        block_mf_gradient = np.asarray([mag_grad_map[k] for k in block_names_list])

        # Full gradient computation: 
        m_full = full_graph_ising_system.mf_magnetization(background_field,beta)
        mag_grad_full = full_graph_ising_system.mf_sparse_magnetisation_gradient(m_full, beta)

        # Compute the controls:
        H_full = np.sum(mag_grad_full)
        H_block = np.sum(block_mf_gradient)
        uniform_control = (Field_Budget / len(graph)) * np.ones(len(graph))
        mag_grad_full_at_field_budget = (Field_Budget/H_full)*mag_grad_full
        mag_grad_block_at_field_budget = (Field_Budget/H_block)*block_mf_gradient
        negative_cancelling_field = []
        for field in background_field :
            if field < 0.0:
                negative_cancelling_field.append(-1.0*field)
            else:
                negative_cancelling_field.append(0.0)
        negative_cancelling_field = np.asarray(negative_cancelling_field)
        negative_cancelling_field = (Field_Budget/np.sum(negative_cancelling_field))*np.asarray(negative_cancelling_field) 
        swing_voter_nodes = np.asarray([check_group(k,'ages_22-28') for k in list(bratislava_profiles['age_group']) ])
        H_SV = np.sum(swing_voter_nodes)
        swing_voter_control =  (Field_Budget/H_SV)*swing_voter_nodes
        
        #Snapshot control
        initial_state = init_sign*np.ones(len(graph))
        block_snapshot = Run_MonteCarlo_Snapshot(relab_graph,groups_node_ids ,0, beta_factor,beta_c ,T_Burn=T_Burn,addition_control=None,sampling_method="Metropolis",full_graph_field=background_field,initial_state=initial_state,frac_to_sample=sample_frac)
        snapshot_gradient = [ (1.0-k**2) for k in block_snapshot ] 
        snapshot_grad_map = { i:j for i,j in zip(block_names,snapshot_gradient)}
        snapshot_influence = np.asarray([snapshot_grad_map[k] for k in block_names_list])
        #Normalize:
        H_snapshot = np.sum(snapshot_influence)
        snapshot_influence = (Field_Budget/H_snapshot)*snapshot_influence
        
        control_dict['no con']=np.zeros(len(graph))
        control_dict['unif']=uniform_control
        control_dict['full']=mag_grad_full_at_field_budget
        control_dict['block']=block_sus_control_field
        control_dict['sv']=swing_voter_control
        control_dict['nc']=negative_cancelling_field
        control_dict['snapshot']=snapshot_influence
        
        # 'full','block','sv','nc','snapshot'
        
        
        for s in tqdm.tqdm_notebook( range(Samples) )  : 

            
            # initial_state = init_sign*np.ones(len(graph))
            # no_control = Run_MonteCarlo_Average(relab_graph, T, beta_factor,beta_c, T_Burn=T_Burn,addition_control=None,sampling_method="Metropolis",full_graph_field=background_field,initial_state=initial_state)

            magnetisations_dict={}
            for con_name in con_names : 
                initial_state = init_sign*np.ones(len(graph))
                control_w_background=np.asarray([i+j for i,j in zip(control_dict[con_name],background_field)])
                magnetisations=Run_MonteCarlo_Average(relab_graph, T, beta_factor,beta_c, T_Burn=T_Burn,addition_control=None,sampling_method="Metropolis",full_graph_field=control_w_background,initial_state=initial_state)
                magnetisations_dict[con_name]=magnetisations
            
            """
            
            initial_state = init_sign*np.ones(len(graph))
            unif_control_field = np.asarray([i+j for i,j in zip(uniform_control,background_field)])
            unif_control = Run_MonteCarlo_Average(relab_graph, T, beta_factor,beta_c, T_Burn=T_Burn,addition_control=None,sampling_method="Metropolis",full_graph_field=unif_control_field,initial_state=initial_state)

            initial_state = init_sign*np.ones(len(graph))
            full_sus_control_field = np.asarray([i+j for i,j in zip(mag_grad_full_at_field_budget,background_field)])
            full_sus_control_mags = Run_MonteCarlo_Average(relab_graph, T, beta_factor,beta_c, T_Burn=T_Burn,addition_control=None,sampling_method="Metropolis",full_graph_field=full_sus_control_field,initial_state=initial_state)

            initial_state = init_sign*np.ones(len(graph))
            block_sus_control_field = np.asarray([i+j for i,j in zip(mag_grad_block_at_field_budget,background_field)])
            block_sus_control_mags = Run_MonteCarlo_Average(relab_graph, T, beta_factor,beta_c, T_Burn=T_Burn,addition_control=None,sampling_method="Metropolis",full_graph_field=block_sus_control_field ,initial_state=initial_state)

            initial_state = init_sign*np.ones(len(graph))
            sv_control_field = np.asarray([i+j for i,j in zip(swing_voter_control,background_field)])
            sv_control_mags = Run_MonteCarlo_Average(relab_graph, T, beta_factor,beta_c, T_Burn=T_Burn,addition_control=None,sampling_method="Metropolis",full_graph_field=sv_control_field  ,initial_state=initial_state)

            initial_state = init_sign*np.ones(len(graph))
            nc_control_field = np.asarray([i+j for i,j in zip(negative_cancelling_field,background_field)])
            nc_control_mags = Run_MonteCarlo_Average(relab_graph, T, beta_factor,beta_c, T_Burn=T_Burn,addition_control=None,sampling_method="Metropolis",full_graph_field=nc_control_field  ,initial_state=initial_state)

            initial_state = init_sign*np.ones(len(graph))
            snapshot_control_field = np.asarray([i+j for i,j in zip(snapshot_influence,background_field)])
            snapshot_control_mags = Run_MonteCarlo_Average(relab_graph, T, beta_factor,beta_c, T_Burn=T_Burn,addition_control=None,sampling_method="Metropolis",full_graph_field=snapshot_control_field  ,initial_state=initial_state)
            
            """
            
            
            
            means=[]
            ses=[]
            #Loop in same order as cont_names
            # control_list = [no_control,unif_control,full_sus_control_mags,block_sus_control_mags,sv_control_mags,nc_control_mags,snapshot_control_mags]
            #for k in control_list :
            for k in list(magnetisations_dict.values())
                print(mean_and_se(k))
                means.append(np.mean(k))
                ses.append(stats.sem(k))

            eval_data = eval_data.append(pd.DataFrame({'control':list(magnetisations_dict.keys()),'magnetisation':means,'mag_se':ses,'beta_factor':beta_factor*np.ones(len(means)),'gradient':gradient*np.ones(len(means))}))
            #eval_data.to_csv("Data/Pokec_control_eval_data_positive.csv")
            #eval_data.to_csv("Data/Pokec_control_eval_data_beta2_neg.csv")
            #eval_data.to_csv("Data/Pokec_control_eval_data_no_field.csv")
            #eval_data.to_csv("Data/Pokec_control_eval_data_g5_beta_impact.csv")
            #eval_data.to_csv("Data/Pokec_control_eval_data_negative_beta_8_start_neg_g.csv")
            #eval_data.to_csv("Data/Pokec_control_eval_data_positive_larger_gs.csv")
            #eval_data.to_csv("Data/Pokec_control_eval_data_positive_start_neg_g.csv")
            #eval_data.to_csv(f"Data/Pokec_control_eval_data_beta_{beta_factor}_{init_sign}".replace('.','-') + ".csv")



-0.05109613278344939 +/- 2.0082654455258736e-05 
-0.0370499357717531 +/- 2.6259960448718197e-05 
-0.016639686295720373 +/- 1.6815778578162947e-05 
-0.03800378608613346 +/- 2.4654953165150715e-05 
-0.04728492326414712 +/- 2.3662376507910086e-05 
-0.03712792238523427 +/- 1.5852492254867876e-05 
-0.038737434926644575 +/- 1.2963529885786536e-05 



-0.05854331012101954 +/- 1.5152786522443364e-05 
-0.022185653437901427 +/- 9.066167273304621e-06 
-0.029462659725508756 +/- 1.0016442564205993e-05 
-0.03258937191535393 +/- 3.55272268741399e-05 
-0.05453005205868433 +/- 4.195973650417218e-05 
-0.043392928131972146 +/- 1.6637438160327712e-05 
-0.05298115746061794 +/- 2.0079700684935385e-05 



-0.06030354945575012 +/- 2.7877805715141613e-05 
-0.030354181596917044 +/- 1.4343588067456835e-05 
-0.017353674531809884 +/- 2.6599209300736255e-05 
-0.03309528767493746 +/- 2.737350987642153e-05 
-0.035089378676222034 +/- 1.9016862632352965e-05 
-0.04708453789466568 +/- 4.2734567823964586e-05 
-0.033064370225136905 +/- 1.4147763136347316e-05 



-0.059426428233385176 +/- 1.978427749301847e-05 
-0.03997917652626597 +/- 1.0410948546485214e-05 
-0.031334453383814485 +/- 1.0889458721170763e-05 
-0.04463056588465959 +/- 8.381998406548637e-06 
-0.03578471367723616 +/- 1.9035337673544555e-05 
-0.04806410655128118 +/- 7.681419908657912e-06 
-0.034112041106078024 +/- 2.9274347008255777e-05 



-0.07661688864850247 +/- 1.690932930389726e-05 
-0.03728883780677439 +/- 3.549299064524712e-05 
-0.045832911905888715 +/- 1.9520836444200378e-05 
-0.044025576364005135 +/- 1.5431630175934315e-05 
-0.04259221824082212 +/- 3.4273087961162986e-05 
-0.03753253329727537 +/- 1.3867427518379935e-05 
-0.03807553917923061 +/- 2.7019143290062365e-05 


...


0.22637760124399975 +/- 1.4652459280510846e-05 
0.24794615644648774 +/- 2.291696555324787e-05 
0.2585956460009465 +/- 1.1195926604762287e-05 
0.2491406666215942 +/- 1.756561663143101e-05 
0.2334199918869583 +/- 1.4459926113792527e-05 
0.2527021296734501 +/- 3.72352350333526e-05 
0.24872912581975526 +/- 1.5999614662141776e-05 



0.21907774998309784 +/- 1.3724658294014341e-05 
0.24723741464404028 +/- 1.9629599111133812e-05 
0.24441907240889732 +/- 2.493073268930037e-05 
0.2382485971198702 +/- 9.143460229339455e-06 
0.25002984247177334 +/- 1.7518774002237395e-05 
0.23548220539517273 +/- 1.4605134962093833e-05 
0.2568196606044216 +/- 1.971040959175055e-05 



0.22292927455885336 +/- 3.102805216374455e-05 
0.23190707186802784 +/- 1.0431298643868973e-05 
0.25184581840308295 +/- 2.7332928737661834e-05 
0.2557984923264147 +/- 2.412069523827602e-05 
0.2506173010614563 +/- 2.6829570353762842e-05 
0.2644422621864647 +/- 1.7498635205410604e-05 
0.24868063687377462 +/- 1.4963231058398835e-05 



0.23740423230342775 +/- 2.4007133120007834e-05 
0.23156142248664727 +/- 2.3818760839970746e-05 
0.24924521668582247 +/- 2.0920360665536153e-05 
0.24080696369413837 +/- 1.6336738204456636e-05 
0.25731615171388006 +/- 2.906629590856141e-05 
0.2411860928943276 +/- 1.5319640350592274e-05 
0.24618682982895007 +/- 1.5084236097117632e-05 



0.2365491447501859 +/- 2.588886727220875e-05 
0.2489087012372389 +/- 1.3001544649818747e-05 
0.24567531607058346 +/- 1.1934790477988e-05 
0.2485 +/- 2.0967754706595915e-05 
0.25368725576364 +/- 2.3298966744875944e-05 
0.24912749645054424 +/- 2.4396014460116076e-05 
0.24295351227097559 +/- 2.3023601247772455e-05 



